<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=85cb33ee3b7cebd1c4cdd0319135581cec6122812a30ffb4f7dd2a9d52c05336
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-10 10:40:14
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  11.04 L
Current:  1.42 C
-------------------
Today PnL: 1.26 L (0.9%)
Current PnL: -17.57 L (-11.82%)
CY Booked + Current PnL: -6.54 L (-4.4%)
-------------------
Total profit:  2.02 L
Total loss:  -19.59 L
-------------------
Total Booked + Current PnL: 20.44 L (16.8%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.79%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 77.22 L (54.51%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.52%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,APOLLOHOSP,8285.00,-13.42,61.0,H-LC,11.16,173008.0,16831.0,9256.0,0.49,10.78,5.35,16.71,22.0,1.82,1.32,30.09,X40N,BTT,HEALTHCARE
62,SBIN,863.00,-12.30,59.0,M-LC,4.30,217817.0,16338.0,10869.0,1.62,8.11,4.99,13.51,60.0,1.50,1.67,21.73,XY25,NTT,BANKS
77,TTKPRESTIG,770.00,104.70,60.0,M-SC,5.58,88189.0,-12588.0,12682.0,-0.81,-12.49,14.38,0.09,245.0,-0.99,0.68,15.86,OX40N,NTT,DURABLES
49,NATIONALUM,247.44,-44.49,68.0,H-MC,2.86,111353.0,10849.0,19787.0,1.68,10.79,17.77,30.49,79.0,0.55,0.85,46.70,MH,ATH,METALS
41,ITC,452.00,-37.92,50.0,H-LC,3.30,200319.0,181.0,20713.0,-0.10,0.09,10.34,10.44,4.0,0.01,1.53,5.66,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,42.74,62.0,M-MC,7.08,225479.0,517.0,167148.0,1.15,0.23,74.13,74.53,192.0,0.00,1.73,32.56,XY24,BTT,STEEL
35,ICICIPRULI,790.00,-20.65,43.0,H-MC,1.99,136481.0,693.0,42063.0,0.96,0.51,30.82,31.48,107.0,0.02,1.04,12.69,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.48,63.0,H-MC,1.73,248976.0,1245.0,73821.0,0.56,0.50,29.65,30.30,92.0,0.02,1.91,13.86,X40,ATH,ELECTRICAL
34,ICICIGI,2260.25,-20.11,45.0,H-MC,3.49,137418.0,1285.0,29847.0,1.20,0.94,21.72,22.86,91.0,0.04,1.05,13.68,X40,ATH,INSURANCE
12,BANKINDIA,190.00,-30.43,59.0,H-MC,5.50,178177.0,-1631.0,114051.0,2.73,-0.91,64.01,62.52,88.0,-0.01,1.36,32.87,XR,NTT,BANKS


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-15.98,39.0,H-LC,6.66,302731.0,-13111.0,125785.0,0.09,-4.15,41.55,35.67,5.0,-0.10,2.32,9.13,X40N,ATH,FMCG
43,JIOFIN,387.00,-1.23,44.0,H-LC,13.77,218435.0,1029.0,52468.0,1.33,0.47,24.02,24.61,48.0,0.02,1.67,55.55,XY24,BTT,FINANCE
80,UNITDSPR,1644.00,-14.23,46.0,H-LC,8.25,172616.0,-9143.0,44397.0,0.25,-5.03,25.72,19.39,86.0,-0.21,1.32,2.71,X40N,NTT,BREWERIES
31,HCLTECH,1943.91,0.65,48.0,H-LC,10.16,229895.0,-12029.0,75291.0,2.61,-4.97,32.75,26.15,8.0,-0.16,1.76,8.64,X40,ATH,IT
56,RELIANCE,1533.00,-14.04,48.0,H-LC,6.50,215561.0,5195.0,23582.0,0.41,2.47,10.94,13.68,37.0,0.22,1.65,19.40,XY25,NTT,REFINERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-2.60,62.0,H-LC,1.30,241808.0,-4343.0,83545.0,3.63,-1.76,34.55,32.18,16.0,-0.05,1.85,33.64,X200,ATH,IT
50,NESTLEIND,1377.0,-8.48,59.0,H-LC,2.29,281502.0,16076.0,40705.0,0.07,6.06,14.46,21.40,11.0,0.39,2.16,13.10,XY25,NTT,FMCG
41,ITC,452.0,-37.92,50.0,H-LC,3.30,200319.0,181.0,20713.0,-0.10,0.09,10.34,10.44,4.0,0.01,1.53,5.66,X40,NTT,FMCG
20,CIPLA,1795.0,-19.44,54.0,H-LC,5.96,214597.0,10097.0,31310.0,0.95,4.94,14.59,20.25,10.0,0.32,1.64,14.10,X40N,BTT,PHARMA
56,RELIANCE,1533.0,-14.04,48.0,H-LC,6.50,215561.0,5195.0,23582.0,0.41,2.47,10.94,13.68,37.0,0.22,1.65,19.40,XY25,NTT,REFINERIES


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1963.11,-18.69,75.0,H-SC,8.38,185845.0,-53570.0,326530.0,5.36,-22.38,175.70,114.01,133.0,-0.16,1.42,70.45,AR,ATH,IT
72,TATAELXSI,9161.00,-18.33,60.0,H-MC,8.20,81473.0,-21179.0,46782.0,4.79,-20.63,57.42,24.94,98.0,-0.45,0.62,23.85,OX40N,NTT,IT
76,TRIDENT,48.00,-6.93,60.0,M-SC,5.98,72519.0,-19736.0,46042.0,4.45,-21.39,63.49,28.51,224.0,-0.43,0.56,24.09,XR,NTT,TEXTILES
57,REPCOHOME,880.00,-60.51,49.0,H-SC,8.18,146544.0,-58454.0,203696.0,4.13,-28.51,139.00,70.85,134.0,-0.29,1.12,20.39,XY24,NTT,FINANCE
79,UNIONBANK,163.00,-16.28,52.0,M-LC,8.96,150391.0,9551.0,34936.0,3.95,6.78,23.23,31.59,66.0,0.27,1.15,35.07,XY24,NTT,BANKS


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VALIANTORG,838.00,-330.19,58.0,H-SC,8.69,97760.0,-35526.0,120118.0,-3.52,-26.65,122.87,63.47,139.0,-0.30,0.75,63.87,XR,NTT,CHEMICALS
46,LAOPALA,464.00,116.69,52.0,H-SC,3.06,68815.0,-32031.0,57860.0,-1.95,-31.76,84.08,25.61,142.0,-0.55,0.53,27.59,AR,NTT,CERAMICS
8,ATULAUTO,844.00,3667.57,61.0,M-SC,7.62,117434.0,-29934.0,81746.0,-1.39,-20.31,69.61,35.16,236.0,-0.37,0.90,19.95,XY24,NTT,AUTO
51,PGHH,17973.08,-31.21,50.0,H-MC,4.71,199215.0,-1605.0,70383.0,-0.81,-0.80,35.33,34.25,80.0,-0.02,1.53,4.36,X40,ATH,FMCG
77,TTKPRESTIG,770.00,104.70,60.0,M-SC,5.58,88189.0,-12588.0,12682.0,-0.81,-12.49,14.38,0.09,245.0,-0.99,0.68,15.86,OX40N,NTT,DURABLES


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,105.37,70.0,H-SC,12.81,129735.0,-12744.0,32265.0,1.22,-8.94,24.87,13.70,163.0,-0.39,0.99,53.12,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,104.70,60.0,M-SC,5.58,88189.0,-12588.0,12682.0,-0.81,-12.49,14.38,0.09,245.0,-0.99,0.68,15.86,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.24,65.0,H-SC,2.35,225630.0,-44037.0,80369.0,0.82,-16.33,35.62,13.47,138.0,-0.55,1.73,14.12,XY24,NTT,PAINTS
37,INDIGOPNTS,1408.0,114.29,49.0,M-SC,3.02,140616.0,-33943.0,33973.0,0.56,-19.44,24.16,0.02,221.0,-1.00,1.08,22.31,OX40N,NTT,PAINTS
19,CERA,9475.0,-19.81,46.0,H-SC,2.28,114300.0,-29503.0,56247.0,0.09,-20.52,49.21,18.60,149.0,-0.52,0.88,26.13,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4998.00,-26.44,53.0,H-LC,13.93,293205.0,-52751.0,176597.0,2.29,-15.25,60.23,35.80,1.0,-0.30,2.24,3.87,X40,BTT,IT
39,INFY,2275.00,-16.37,58.0,H-LC,10.11,326508.0,13620.0,158063.0,1.90,4.35,48.41,54.87,3.0,0.09,2.50,11.29,X40,BTT,IT
41,ITC,452.00,-37.92,50.0,H-LC,3.30,200319.0,181.0,20713.0,-0.10,0.09,10.34,10.44,4.0,0.01,1.53,5.66,X40,NTT,FMCG
83,VBL,671.64,-15.98,39.0,H-LC,6.66,302731.0,-13111.0,125785.0,0.09,-4.15,41.55,35.67,5.0,-0.10,2.32,9.13,X40N,ATH,FMCG
1,ABB,7934.00,-40.55,50.0,H-LC,12.53,248544.0,-13075.0,132300.0,0.90,-5.00,53.23,45.57,7.0,-0.10,1.90,5.76,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,42.74,62.0,M-MC,7.08,225479.0,517.0,167148.0,1.15,0.23,74.13,74.53,192.0,0.00,1.73,32.56,XY24,BTT,STEEL
4,ANGELONE,3033.00,9.18,42.0,H-SC,6.37,192913.0,1907.0,55790.0,0.71,1.00,28.92,30.21,157.0,0.03,1.48,22.09,X40N,NTT,FINANCE
15,BLUESTARCO,2326.38,8.87,52.0,H-MC,3.53,188000.0,23330.0,44631.0,-0.37,14.17,23.74,41.28,89.0,0.52,1.44,23.28,X40N,ATH,AC
84,VOLTAS,1918.49,0.74,58.0,H-MC,3.30,211410.0,19668.0,76361.0,-0.21,10.26,36.12,50.08,99.0,0.26,1.62,17.75,XY25,ATH,AC
30,HAVELLS,2069.16,0.48,63.0,H-MC,1.73,248976.0,1245.0,73821.0,0.56,0.50,29.65,30.30,92.0,0.02,1.91,13.86,X40,ATH,ELECTRICAL


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-44.49,68.0,H-MC,2.86,111353.0,10849.0,19787.0,1.68,10.79,17.77,30.49,79.0,0.55,0.85,46.70,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-39.65,60.0,M-SC,2.51,102045.0,10546.0,68207.0,-0.77,11.53,66.84,86.07,223.0,0.15,0.78,48.19,XR,NTT,DURABLES
36,INDIAMART,4850.92,-50.91,50.0,H-SC,12.15,136804.0,13468.0,120292.0,0.25,10.92,87.93,108.45,119.0,0.11,1.05,35.71,AR,ATH,MISC
86,WIPRO,420.00,-9.71,60.0,M-LC,6.48,158708.0,7763.0,101700.0,2.74,5.14,64.08,72.51,53.0,0.08,1.22,11.47,XR,NTT,IT
12,BANKINDIA,190.00,-30.43,59.0,H-MC,5.50,178177.0,-1631.0,114051.0,2.73,-0.91,64.01,62.52,88.0,-0.01,1.36,32.87,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6705.56,51.0,L-SC,6.08,74652.0,-19158.0,95779.0,-0.08,-20.42,128.30,81.67,269.0,-0.20,0.57,44.57,XR,NTT,CERAMICS
3,ALKYLAMINE,4546.37,-4.34,46.0,H-SC,14.69,89014.0,-11949.0,106479.0,1.99,-11.84,119.62,93.63,148.0,-0.11,0.68,32.58,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-39.64,59.0,L-SC,42.18,90240.0,-23309.0,63358.0,0.66,-20.53,70.21,35.27,268.0,-0.37,0.69,120.86,XR,NTT,HOTELS
48,MASFIN,398.61,-15.99,61.0,H-SC,13.03,95640.0,-2340.0,23939.0,-0.09,-2.39,25.03,22.05,152.0,-0.10,0.73,39.02,XR,ATH,FINANCE
82,VALIANTORG,838.00,-330.19,58.0,H-SC,8.69,97760.0,-35526.0,120118.0,-3.52,-26.65,122.87,63.47,139.0,-0.30,0.75,63.87,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.0,-39.64,59.0,L-SC,42.18,90240.0,-23309.0,63358.0,0.66,-20.53,70.21,35.27,268.0,-0.37,0.69,120.86,XR,NTT,HOTELS
60,SAMMAANCAP,326.0,-171.59,65.0,M-SC,3.89,83004.0,-19206.0,112595.0,1.48,-18.79,135.65,91.37,208.0,-0.17,0.64,34.74,XY25,NTT,FINANCE
64,SHALBY,327.0,1108.93,77.0,M-SC,2.29,164324.0,-16819.0,62279.0,2.49,-9.29,37.90,25.10,235.0,-0.27,1.26,31.69,XY24,NTT,HEALTHCARE
8,ATULAUTO,844.0,3667.57,61.0,M-SC,7.62,117434.0,-29934.0,81746.0,-1.39,-20.31,69.61,35.16,236.0,-0.37,0.90,19.95,XY24,NTT,AUTO
75,TITAGARH,1548.0,-3.20,64.0,H-SC,7.11,158646.0,-33369.0,112258.0,2.56,-17.38,70.76,41.08,158.0,-0.30,1.21,34.43,XY24,NTT,ENGINEERING


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.36
1,25,44.47
2,50,75.11


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.92
LC    35.38
MC    23.70
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.71
X40      16.05
X40N     13.08
XY25     10.28
XR        9.53
AR        8.37
OX40N     7.77
X200      1.85
MH        1.79
X5K       1.45
SR        1.12
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.79
H-SC    25.08
H-MC    20.61
M-SC    14.18
M-LC     7.48
M-MC     2.76
L-SC     1.66
L-LC     1.11
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.04,-14.43,72.37
FMCG,12.24,-3.85,39.06
FINANCE,9.10,-17.51,59.76
BANKS,7.75,-12.52,60.30
PAINTS,6.18,-14.63,40.29
MISC,6.00,-17.05,84.14
ELECTRICAL,5.86,-7.48,66.73
AC,3.81,2.54,30.46
AUTO,3.49,-15.17,67.11


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2713130.0,22
AR,1177564.0,9
XR,1049947.0,13
X40,869497.0,10
X40N,565910.0,10
OX40N,445014.0,10
XY25,410541.0,7
SR,196732.0,2
X5K,128099.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2749184.0,28
M-SC,1395379.0,17
H-LC,1364011.0,15
H-MC,1150673.0,15
M-LC,402008.0,5
M-MC,317270.0,2
L-SC,243687.0,3
L-MC,59643.0,1
L-LC,39795.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,907508.0,6
M-SC,XY24,812927.0,7
H-SC,AR,646513.0,5
H-LC,X40,581935.0,5
H-SC,XR,501167.0,6
H-MC,XY24,461617.0,4
H-LC,AR,371028.0,2
M-MC,XY24,317270.0,2
H-SC,X40N,245835.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
